In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from config import g_key

In [3]:
# Read .csv file created from Merged Power plant data
PowerPlantData = pd.read_csv("OutputData/Merged_Power_Plant_Data.csv")
PowerPlantData_CountryStatus = pd.read_csv("OutputData/Power_Plant_Data_Country_Status.csv")

In [4]:
# Display Power Plant data frame
PowerPlantData.head()

,Country Code,Country Name,Power Plant Name,Power Plant Capacity(MWh),Latitude,Longitude,Primary Fuel,GNI,Population
0,AFG,Afghanistan,Kajaki Hydroelectric Power Plant Afghanistan,33.0,32.322,65.1190,Hydro,500.0,38928341.0
1,AFG,Afghanistan,Kandahar DOG,10.0,31.670,65.7950,Solar,500.0,38928341.0
2,AFG,Afghanistan,Kandahar JOL,10.0,31.623,65.7920,Solar,500.0,38928341.0
3,AFG,Afghanistan,Mahipar Hydroelectric Power Plant Afghanistan,66.0,34.556,69.4787,Hydro,500.0,38928341.0
4,AFG,Afghanistan,Naghlu Dam Hydroelectric Power Plant Afghanistan,100.0,34.641,69.7170,Hydro,500.0,38928341.0


In [5]:
# Add a new column to calculate the MW capacity for a year 
# (so it's a more manageable number to work with when divided by the population)
# There are 8760 hours in a year.
PowerPlantData["Power Plant Capacity(MWh per Year)"] = ((PowerPlantData["Power Plant Capacity(MWh)"]) * (8760))
PowerPlantData

,Country Code,Country Name,Power Plant Name,Power Plant Capacity(MWh),Latitude,Longitude,Primary Fuel,GNI,Population,Power Plant Capacity(MWh per Year)
0,AFG,Afghanistan,Kajaki Hydroelectric Power Plant Afghanistan,33.0,32.3220,65.1190,Hydro,500.0,38928341.0,289080.0
1,AFG,Afghanistan,Kandahar DOG,10.0,31.6700,65.7950,Solar,500.0,38928341.0,87600.0
2,AFG,Afghanistan,Kandahar JOL,10.0,31.6230,65.7920,Solar,500.0,38928341.0,87600.0
3,AFG,Afghanistan,Mahipar Hydroelectric Power Plant Afghanistan,66.0,34.5560,69.4787,Hydro,500.0,38928341.0,578160.0
4,AFG,Afghanistan,Naghlu Dam Hydroelectric Power Plant Afghanistan,100.0,34.6410,69.7170,Hydro,500.0,38928341.0,876000.0
...,...,...,...,...,...,...,...,...,...,...
34931,ZMB,Zambia,Ndola,50.0,-12.9667,28.6333,Oil,1160.0,18383956.0,438000.0
34932,ZMB,Zambia,Nkana,20.0,-12.8167,28.2000,Oil,1160.0,18383956.0,175200.0
34933,ZMB,Zambia,Victoria Falls,108.0,-17.9167,25.8500,Hydro,1160.0,18383956.0,946080.0
34934,ZWE,Zimbabwe,Hwange Coal Power Plant Zimbabwe,920.0,-18.3835,26.4700,Coal,1140.0,14862927.0,8059200.0


In [6]:
# Display datatypes
PowerPlantData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34936 entries, 0 to 34935
Data columns (total 10 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Country Code                        34936 non-null  object 
 1   Country Name                        34936 non-null  object 
 2   Power Plant Name                    34936 non-null  object 
 3   Power Plant Capacity(MWh)           34936 non-null  float64
 4   Latitude                            34936 non-null  float64
 5   Longitude                           34936 non-null  float64
 6   Primary Fuel                        34936 non-null  object 
 7   GNI                                 34755 non-null  float64
 8   Population                          34886 non-null  float64
 9   Power Plant Capacity(MWh per Year)  34936 non-null  float64
dtypes: float64(6), object(4)
memory usage: 2.7+ MB


In [7]:
# Display Power Plant Country Status data frame
PowerPlantData_CountryStatus.head()

,Country Code,Country Name,Power Plant Name,Power Plant Capacity(MWh),Latitude,Longitude,Primary Fuel,GNI,Population,Country Status
0,AFG,Afghanistan,Kajaki Hydroelectric Power Plant Afghanistan,33.0,32.322,65.1190,Hydro,500.0,38928341.0,Developing
1,AFG,Afghanistan,Kandahar DOG,10.0,31.670,65.7950,Solar,500.0,38928341.0,Developing
2,AFG,Afghanistan,Kandahar JOL,10.0,31.623,65.7920,Solar,500.0,38928341.0,Developing
3,AFG,Afghanistan,Mahipar Hydroelectric Power Plant Afghanistan,66.0,34.556,69.4787,Hydro,500.0,38928341.0,Developing
4,AFG,Afghanistan,Naghlu Dam Hydroelectric Power Plant Afghanistan,100.0,34.641,69.7170,Hydro,500.0,38928341.0,Developing


In [8]:
# Filter Country Status by Developed Countries
DevelopedPowerPlant = PowerPlantData_CountryStatus[PowerPlantData_CountryStatus["Country Status"] == "Developed"]

#Display data frame
DevelopedPowerPlant.head(5)

,Country Code,Country Name,Power Plant Name,Power Plant Capacity(MWh),Latitude,Longitude,Primary Fuel,GNI,Population,Country Status
336,AUS,Australia,Adelaide Showgrounds,1.0,-34.9453,138.5882,Solar,53690.0,25687041.0,Developed
337,AUS,Australia,Albany Wind Farm,21.6,-35.0641,117.7977,Wind,53690.0,25687041.0,Developed
338,AUS,Australia,Amcor Gawler,4.0,-34.5509,138.7659,Oil,53690.0,25687041.0,Developed
339,AUS,Australia,Angaston,50.0,-34.5034,139.0246,Oil,53690.0,25687041.0,Developed
340,AUS,Australia,Anglesea,165.0,-38.3861,144.1828,Coal,53690.0,25687041.0,Developed


In [9]:
# Save data frame as csv file
DevelopedPowerPlant.to_csv('OutputData/Power_Plant_Data_Developed.csv', index = False)

In [10]:
# Filter Country Status by Developing Countries
DevelopingPowerPlant = PowerPlantData_CountryStatus[PowerPlantData_CountryStatus["Country Status"] == "Developing"]

#Display data frame
DevelopingPowerPlant.head(5)

,Country Code,Country Name,Power Plant Name,Power Plant Capacity(MWh),Latitude,Longitude,Primary Fuel,GNI,Population,Country Status
0,AFG,Afghanistan,Kajaki Hydroelectric Power Plant Afghanistan,33.0,32.322,65.1190,Hydro,500.0,38928341.0,Developing
1,AFG,Afghanistan,Kandahar DOG,10.0,31.670,65.7950,Solar,500.0,38928341.0,Developing
2,AFG,Afghanistan,Kandahar JOL,10.0,31.623,65.7920,Solar,500.0,38928341.0,Developing
3,AFG,Afghanistan,Mahipar Hydroelectric Power Plant Afghanistan,66.0,34.556,69.4787,Hydro,500.0,38928341.0,Developing
4,AFG,Afghanistan,Naghlu Dam Hydroelectric Power Plant Afghanistan,100.0,34.641,69.7170,Hydro,500.0,38928341.0,Developing


In [11]:
# Save data frame as csv file
DevelopingPowerPlant.to_csv('OutputData/Power_Plant_Data_Developing.csv', index = False)

In [12]:
# Assign the marker layer to a variable
locations = PowerPlantData[["Latitude", "Longitude"]]
fig = gmaps.figure(map_type="SATELLITE",
    center=[17.133932434766733, 16.103938729508073],
    zoom_level=1.6)
markers = gmaps.marker_layer(locations)

fig.add_layer(markers)

fig

Figure(layout=FigureLayout(height='420px'))